# (나의) 구글 Location History 데이터 지도에 표시하기

-------------------------------

### 지도 라이브러리

In [ ]:
!pip install folium

In [ ]:
import folium

folium.__version__

In [3]:
import folium

#------------------
# 중심위치 [위도,경도]=[Latitude, longitude] 
#------------------
m = folium.Map(location = [37.549328, 126.8513969],
               zoom_start=16)

# 마커 표시하기
folium.Marker(location = [37.549328, 126.8513969],
              popup = '강서여성인력개발센터', 
              icon=folium.Icon(color='pink', icon='info-sign')
).add_to(m)
m

## [실습] (나의) 구글 Location History 데이터 지도에 표시하기


### 1. 구글 Location History 데이터 다운로드
- 구글 크롬 브라우저에 로그인 된 상태에서
- https://takeout.google.com/settings/takeout 사이트로 이동하기
- 모두 체크를 해제시키고 **[위치 기록]** 만 체크표시하고 
- takeout~.zip 파일이 다운로드 된다.

### 2. 필요한 라이브러리 설치하기
설치시 오류가 발생하면 윈도우 명령프롬프트창에서 timezonefinder 설치를 진행해 본다.

In [ ]:
import numpy as np

np.__version__

In [ ]:
!pip install timezonefinder

In [6]:
# 경도, 위도값으로 시간대 정보 가져오기
from timezonefinder import TimezoneFinder

tzf= TimezoneFinder()
tz = tzf.timezone_at(lng=128, lat=38)  # 'Asia/Seoul'
#tz = tzf.timezone_at(lng=13.358, lat=52.5061)  # 'Europe/Berlin'
tz

'Asia/Seoul'

### 3.나의 Local History데이터 정보를 딕셔너리타입으로 변환하기

In [9]:
from zipfile import ZipFile
import pandas as pd
import json
from timezonefinder import TimezoneFinder

In [10]:
# Google Location History takeout file
in_file = "data/takeout-20240415T132831Z-001.zip"
out_file = "data/place_visits.csv"

# store all places into this array
place_visits = []

with ZipFile(in_file) as myzip:
    for file in myzip.filelist[:]:
        filename = file.filename
        
        if "Semantic Location History" in filename:
            # process all files in "Semantic Location History" directory
            history_json = json.load(myzip.open(filename))

            for timeline_object in history_json["timelineObjects"]:
               
                if "placeVisit" in timeline_object:
                    place_visit_json = timeline_object["placeVisit"]
                    
                    # 위도.경도 값 없으면 skip
                    if not "location" in place_visit_json or not "latitudeE7" in place_visit_json["location"]:
                        continue
                    
                    place_visit = {
                        "placeId": place_visit_json["location"]["placeId"],
                        "locationConfidence": place_visit_json["location"]["locationConfidence"],
                        "startTimestamp": place_visit_json["duration"]["startTimestamp"],
                        "endTimestamp": place_visit_json["duration"]["endTimestamp"],
                        "placeVisitImportance": place_visit_json["placeVisitImportance"],
                        "placeVisitType": place_visit_json["placeVisitType"],
                        "latitudeE7": place_visit_json["location"]["latitudeE7"],
                        "longitudeE7": place_visit_json["location"]["longitudeE7"],
                    }
                    
                    for optional_field in ["centerLatE7", "centerLngE7"]:
                        if optional_field in place_visit_json:
                            place_visit[optional_field] = place_visit_json[optional_field]
                        else:
                            place_visit[optional_field] = None
                    
                    for optional_field in ["name", "address"]:
                        if optional_field in place_visit_json["location"]:
                            place_visit[optional_field] = place_visit_json["location"][optional_field]
                        else:
                            place_visit[optional_field] = None
                        
                    
                    place_visits.append(place_visit)

print(f'레코드 총 개수: {len(place_visits)}')   
place_visits[:3]

레코드 총 개수: 2012


[{'placeId': 'ChIJ72--gersejURkkuplO6dNbY',
  'locationConfidence': 95.89299,
  'startTimestamp': '2023-04-01T03:02:14.732Z',
  'endTimestamp': '2023-04-01T03:23:01.229Z',
  'placeVisitImportance': 'MAIN',
  'placeVisitType': 'SINGLE_PLACE',
  'latitudeE7': 366193904,
  'longitudeE7': 1268465111,
  'centerLatE7': 366193529,
  'centerLngE7': 1268463414,
  'name': 'Yesan Service Area',
  'address': '대한민국 충청남도 예산군 신양면 당진영덕고속도로 33'},
 {'placeId': 'ChIJiWKXIGxhezURSdDzm8-jpb0',
  'locationConfidence': 60.710876,
  'startTimestamp': '2023-04-01T04:48:46.437Z',
  'endTimestamp': '2023-04-02T04:54:45.481Z',
  'placeVisitImportance': 'MAIN',
  'placeVisitType': 'SINGLE_PLACE',
  'latitudeE7': 374308554,
  'longitudeE7': 1268842593,
  'centerLatE7': 374308424,
  'centerLngE7': 1268842023,
  'name': '501 광명역세권휴먼시아5단지',
  'address': '대한민국 광명시 소하2동'},
 {'placeId': 'ChIJDdW3d1dhezUR-thy-K_s0UI',
  'locationConfidence': 46.363525,
  'startTimestamp': '2023-04-02T05:03:23.544Z',
  'endTimestamp': '202

### 4. DataFrame 형태로 변환하기

In [11]:
# 1.DataFrame형태로 변환하기
place_visits_df = pd.DataFrame(place_visits)
place_visits_df.head(2)


# 2.데이터 전처리하기
# convert to datetime type
place_visits_df["startTimestamp"] = pd.to_datetime(place_visits_df["startTimestamp"], format="ISO8601")
place_visits_df["endTimestamp"] = pd.to_datetime(place_visits_df["endTimestamp"], format="ISO8601")


# get geo coordinates as float value
place_visits_df["latitude"] = place_visits_df.latitudeE7/10**7   #(1E7 = 1e7 = 1x10^7 = 10**7)
place_visits_df["longitude"] = place_visits_df.longitudeE7/1e7
place_visits_df["centerLat"] = place_visits_df.centerLatE7/1E7
place_visits_df["centerLng"] = place_visits_df.centerLngE7/1E7
 
# add timezone based on geo coordinates
tf = TimezoneFinder()
place_visits_df["timezone"] = place_visits_df.apply(lambda row: tf.timezone_at(lng=row.longitude, lat=row.latitude), axis=1)

# convert UTC time to local timezone
place_visits_df['startTimestamp_local'] = place_visits_df.apply(lambda row: row.startTimestamp.tz_convert(row.timezone), axis=1)
place_visits_df['endTimestamp_local'] =place_visits_df.apply(lambda row: row.endTimestamp.tz_convert(row.timezone), axis=1)

# remove TZ info from datetime
place_visits_df['startTimestamp_local'] = pd.to_datetime(place_visits_df['startTimestamp_local'].apply(lambda x: x.replace(tzinfo=None)))
place_visits_df['endTimestamp_local'] = pd.to_datetime(place_visits_df['endTimestamp_local'].apply(lambda x: x.replace(tzinfo=None)))

# add datetime parts as a separate column to data frame
for datetime_type in [("year", lambda x: x.year), ("month", lambda x: x.month), ("day", lambda x: x.day), ("hour", lambda x: x.hour), ("minute", lambda x: x.minute), ("weekday", lambda x: x.weekday)]:
    for tztype in ["", "_local"]:
        place_visits_df[f"{datetime_type[0]}{tztype}"] = datetime_type[1](place_visits_df[f"startTimestamp{tztype}"].dt)
        

place_visits_df.drop(columns=["latitudeE7", "longitudeE7", "centerLatE7", "centerLngE7"], inplace=True)
place_visits_df["duration"] = place_visits_df.endTimestamp - place_visits_df.startTimestamp
place_visits_df["duration_minutes"] = place_visits_df.duration.dt.total_seconds()/60


# 3. 파일로 저장하기
place_visits_df.to_csv(out_file, index=False)


# place_visits_df.head()
place_visits_df

,placeId,locationConfidence,startTimestamp,endTimestamp,placeVisitImportance,placeVisitType,name,address,latitude,longitude,...,day,day_local,hour,hour_local,minute,minute_local,weekday,weekday_local,duration,duration_minutes
0,ChIJ72--gersejURkkuplO6dNbY,95.892990,2023-04-01 03:02:14.732000+00:00,2023-04-01 03:23:01.229000+00:00,MAIN,SINGLE_PLACE,Yesan Service Area,대한민국 충청남도 예산군 신양면 당진영덕고속도로 33,36.619390,126.846511,...,1,1,3,12,2,2,5,5,0 days 00:20:46.497000,20.774950
1,ChIJiWKXIGxhezURSdDzm8-jpb0,60.710876,2023-04-01 04:48:46.437000+00:00,2023-04-02 04:54:45.481000+00:00,MAIN,SINGLE_PLACE,501 광명역세권휴먼시아5단지,대한민국 광명시 소하2동,37.430855,126.884259,...,1,1,4,13,48,48,5,5,1 days 00:05:59.044000,1445.984067
2,ChIJDdW3d1dhezUR-thy-K_s0UI,46.363525,2023-04-02 05:03:23.544000+00:00,2023-04-02 05:35:11.379000+00:00,MAIN,SINGLE_PLACE,투썸플레이스 광명중앙대병원점,대한민국 경기도 광명시 덕안 로104번길 17,37.424187,126.886935,...,2,2,5,14,3,3,6,6,0 days 00:31:47.835000,31.797250
3,ChIJiWKXIGxhezURSdDzm8-jpb0,69.203094,2023-04-02 05:56:39.379000+00:00,2023-04-04 03:20:23.672000+00:00,MAIN,SINGLE_PLACE,501 광명역세권휴먼시아5단지,대한민국 광명시 소하2동,37.430855,126.884259,...,2,2,5,14,56,56,6,6,1 days 21:23:44.293000,2723.738217
4,ChIJif2a9EFhezURHkD8egX3ht0,88.744064,2023-04-04 03:30:01.575000+00:00,2023-04-04 05:26:34.963000+00:00,MAIN,SINGLE_PLACE,LOTTE Mall Gwangmyeong,대한민국 경기도 광명시 일직로 17,37.424552,126.884223,...,4,4,3,12,30,30,1,1,0 days 01:56:33.388000,116.556467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007,ChIJiWKXIGxhezURSdDzm8-jpb0,52.405155,2022-12-30 10:07:47.867000+00:00,2022-12-31 02:43:28.635000+00:00,MAIN,SINGLE_PLACE,501 광명역세권휴먼시아5단지,대한민국 광명시 소하2동,37.430855,126.884259,...,30,30,10,19,7,7,4,4,0 days 16:35:40.768000,995.679467
2008,ChIJQVfnHk2XezURyP4-KOkA98c,68.450290,2022-12-31 03:29:59.412000+00:00,2022-12-31 06:31:11.888000+00:00,MAIN,SINGLE_PLACE,솔밭회조개구이,대한민국 인천광역시 중구 덕교동 103-23 22385,37.422120,126.423220,...,31,31,3,12,29,29,5,5,0 days 03:01:12.476000,181.207933
2009,ChIJiWKXIGxhezURSdDzm8-jpb0,31.753450,2022-12-31 07:26:19.712000+00:00,2022-12-31 07:31:46.858000+00:00,MAIN,SINGLE_PLACE,501 광명역세권휴먼시아5단지,대한민국 광명시 소하2동,37.430855,126.884259,...,31,31,7,16,26,26,5,5,0 days 00:05:27.146000,5.452433
2010,ChIJif2a9EFhezURHkD8egX3ht0,80.747444,2022-12-31 07:38:59.252000+00:00,2022-12-31 11:18:28.708000+00:00,MAIN,SINGLE_PLACE,LOTTE Mall Gwangmyeong,대한민국 경기도 광명시 일직로 17,37.424552,126.884223,...,31,31,7,16,38,38,5,5,0 days 03:39:29.456000,219.490933


In [12]:
print(f"시작날짜 최솟값: {place_visits_df.startTimestamp.min()}")
print(f"시작날짜 최댓값: {place_visits_df.startTimestamp.max()}")
print(f"종료날짜 최솟값: {place_visits_df.endTimestamp.min()}")
print(f"종료날짜 최댓값: {place_visits_df.endTimestamp.max()}")

시작날짜 최솟값: 2022-06-16 08:36:59.999000+00:00
시작날짜 최댓값: 2024-04-15 07:46:27+00:00
종료날짜 최솟값: 2022-06-16 23:17:07.624000+00:00
종료날짜 최댓값: 2024-04-15 12:33:02.534000+00:00


### 5. (나의)Location History 지도에 표시하기

In [13]:
import folium
import webbrowser
import pandas as pd
import os

# 파일 불러오기
# df = pd.read_csv(file)
df = place_visits_df


center = [37.4306881, 126.8845086]
# m = folium.Map(location=center ,
#                   zoom_start=6, 
#                   width=750, 
#                   height=500)
m = folium.Map(location=center ,
                  zoom_start=4)

# df에서 위도&경도 정보 지도 Marker에 추가하기
folium.Marker(location=center, 
                popup='우리집', icon=folium.Icon(color='red', icon='star'),
                color='tomato', radius = 50,
).add_to(m)


# 구글 Location History 마커로 표시하기
for i, row in df.iterrows():   
    # 아이콘 색상 지정
    date = row['startTimestamp'].strftime('%Y-%m-%d')  # timestamp -> string타입으로 변환
    if date <= '2022-12-31': 
        icon_color='green'
    elif date < '2023-12-31': 
        icon_color='blue'
    else:
        icon_color='red'  # 최근 년도
    # 마커 추가하기
    folium.Marker(location= [row['latitude'], row['longitude']], 
                  icon=folium.Icon(icon='cloud', color=icon_color),
                  popup=row['name']
    ).add_to(m)
    
m    

#html 파일로 저장하여 불러오기
file = '/data/map_my_google_history.html'
file = os.getcwd().replace('\\',"/") + file # 현재 위치 경로 가져오기
m.save(file) 
webbrowser.get().open(file)

True

--------------------

THE END